In [1]:
using PowerModels
using JuMP
using Ipopt

# Test

## read in the case data

In [2]:
file_path = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/"
case_name = "pglib_opf_case200_activ.m"
network_data_orig = PowerModels.parse_file(file_path*case_name);

[warn | PowerModels]: no active generators found at bus 168, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 197, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 92, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 161, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 165, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 196, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 164, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 78, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 166, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 169, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 79, up

## randomly perturb the load

In [8]:
network_data = deepcopy(network_data_orig)
for (load_id, load) in network_data["load"]
    perturbation_factor = 0.8 + (rand() * 0.4)  # rand() generates a random number in [0, 1]
    load["pd"] *= perturbation_factor
    load["qd"] *= perturbation_factor
end

## create the JuMP model

In [9]:
pm = instantiate_model(network_data, DCPPowerModel, PowerModels.build_opf);
optimizer = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
result = optimize_model!(pm, optimizer=optimizer);
status = result["termination_status"]
obj = result["objective"]
time = result["solve_time"]

0.014887094497680664

In [14]:
result["solution"]

Dict{String, Any} with 7 entries:
  "baseMVA"             => 100.0
  "branch"              => Dict{String, Any}("1"=>Dict{String, Any}("qf"=>NaN, …
  "gen"                 => Dict{String, Any}("24"=>Dict{String, Any}("qg"=>NaN,…
  "multiinfrastructure" => false
  "multinetwork"        => false
  "bus"                 => Dict{String, Any}("1"=>Dict{String, Any}("va"=>-0.13…
  "per_unit"            => true

In [11]:
if status == MOI.LOCALLY_SOLVED || status == MOI.OPTIMAL
    println("The objective value is: ", obj)
    println("The solve time is: ", time)
else
    println("The problem was not solved to optimality, status: ", status)
end

LOCALLY_SOLVED::TerminationStatusCode = 4

## Output the model into .lp files

In [10]:
i = 2
JuMP.write_to_file(pm.model, "dcopf_model_$(i).lp")